## Kafka Producer
Streams English tweets from Twitter with particular hashtags

Requires:
- hashtag_file - a text file with one hashtag per line e.g. '../hashtags/covid.txt' or '../hashtags/blm.txt'

In [ ]:
import json
import sys
from kafka import KafkaProducer
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from urllib3.exceptions import ProtocolError

# Credentials to access Twitter
access_token = "147902175-W7X1zzR9aZkicESXM9nwzIZ7mDUtV6kDsJxCWsFx"
access_token_secret = "nkMaGnwE0uOaa9DdlyWiDo3G90U73ahqdiytjSYbqVrDe"
consumer_key = "oyJIFFCTwtMY1D9f8AhHsMwrI"
consumer_secret = "OztDRlpcqodVsIwzxaTAUUdBzM0QcrD6K95rGPjJiYQM1WjUg7"

In [ ]:
hashtag_file = '../hashtags/covid.txt' # '../hashtags/covid.txt' or '../hashtags/blm.txt'
topic = hashtag_file.replace('../hashtags/', '').replace('.txt', '')

file = open(hashtag_file, 'r')
hashtags = [line.replace('\n', '').strip() for line in file.readlines()]

print(hashtags)

In [ ]:
class StdOutListener(StreamListener):
    def on_data(self, data):

        tstream = json.loads(data)

        tweet = ''
        if 'extended_tweet' in tstream:
            tweet = tstream['extended_tweet']['full_text']
            print('extended')

        elif "retweeted_status" in tstream:
            print('retweet')
            tstream = tstream["retweeted_status"]

            if "extended_tweet" in tweet:
                tweet = tstream["extended_tweet"].get("full_text", tstream.get("text", ""))
            else:
                tweet = tstream.get("text", "")

#         print(tweet)
#         print('----------------------')

        producer.send(topic, tweet)
           
        return True

In [ ]:
# Set up producer
producer = KafkaProducer(bootstrap_servers='172.27.1.16:9092',
                         value_serializer=lambda x: json.dumps(x).encode('utf-8'))


# Get the stream of tweets from twitter and filter them by the hastags from from hashtag_file.
out = StdOutListener()
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
stream = Stream(auth, out)

In [ ]:
while True:
    try:
        stream.filter(languages=["en"], track=hashtags)
    except ProtocolError:
        continue